In [1]:
# Assignment 3 file initialize

'''
Submitter : JESOON KANG, 20170937 Dept.of CSE

Data : 2019. 10. 9

Assignment 3.



Binary classification based on 3 layers neural network

'''



'\nSubmitter : JESOON KANG, 20170937 Dept.of CSE\n\nData : 2019. 10. 9\n\nAssignment 3.\n\n\n\nBinary classification based on 3 layers neural network\n\n'

In [2]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os



In [3]:

#### Section 1. #### This Section is bringed Data_import_ex.py file.

# Image Data import & resize

transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = '../data/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False, num_workers=1)  


validation_data_path = '../data/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=1, shuffle=False, num_workers=1)  

#### Section 1 END ####



In [22]:
#### Section 2 START ####
# This Part includes Sigmoid, Hypothesis, Loss, Predict Functions

# sigmoid Function.
def sigmoid(z) :
    return 1 / (1 + np.exp(-z))

In [5]:
# h(x) Function. hypothesis Func.
def hypothesis(weight,X) :
    theta = np.array(weight)
    z = np.dot(X,theta)
    h = sigmoid(z)
    return h

In [6]:
# Calculate hypothesis through (weight_Array, initial X )

def hypothesis_NN(weight_A, X) :
    h = float
    x = X
    for w in weight_A : # for weight (u,v,w), each u, v, w
        theta = np.array(w)
        z = np.dot(x,theta)
        h = sigmoid(z)
        x = h # this x will be used for calculate next layer
    return h




In [7]:
# Loss Function
def lossFunction(h,y) :
    # if i = 1, it occurs ZeroDivisionError ,
    # So, adjust value to 0.995
    for i in h :
        if i == 1 :
            i = 0.995
    try : 
        ret = (-y * np.log(h) - (1-y) * np.log(1-h)).mean()
    except ZeroDivisionError :
        print("Error")
    finally :
        return (-y * np.log(h) - (1-y) * np.log(1-h)).mean()

In [8]:
# predict Function. if h(x) returns >=0.5, set to 1. other cases, set to 0.
def predict(h, labels) :
    mount = len(h)
    correct = 0
    
    for i in range(0,mount) :
        if h[i] >= 0.5 :
            if labels[i] == 1 :
                correct +=1
        else :
            if labels[i] == 0 :
                correct +=1
    return correct * (1/mount)

#### Section 2 END ####

In [10]:
# Loss Function For NN

def get_error_val(output_Y, target_Y) :
    ret = np.subtract(target_Y,output_Y)**2
    return ret

def backpropagation_back(input_X, weight_list, hidden_layer_list, output_Y, target_Y, learning_rate) :
    err_val = get_error_val(output_Y, target_Y)
    
    
    r_h = np.array(reversed(hidden_layer_list))
    r_w = np.array(reversed(weight_list))
    
    deriv = (output_Y-target_Y) * (output_Y*(1-output_Y)) * (hidden_layer_list[0])
    r_w[0] = r_w - learning_rate*deriv
    
    for i in range(1,len(r_h)) :
        deriv = deriv*hidden_layer_list[i]
        r_w[i] = r_w[i] - learning_rate*deriv
    
    deriv = deriv * input_X
    r_w[len(r_w)-1] = r_w[len(r_w)-2] - learning_rate*deriv # First weight

    weight_list = np.array(reversed(r_w))
    

In [11]:
#### Section 3 START ####
# Section 3 includes Data Pre-processing. ReDesign datasets to easy calculate

# Data reconstruct. vectorize

# Each Image file will be stored shape of a row
training_vectorized = []
training_labels = []

validation_vectorized = []
validation_labels = []


# Training data vectorizing
for i, data in enumerate(trainloader) :
    train_data = []
    inputs, labels = data

    for u in inputs :
        for col in u[0] :
            train_data += list(col)
    training_vectorized.append(train_data)
    training_labels.append([labels])

training_vectorized = np.array(training_vectorized)
training_labels = np.array(training_labels)

# Validation data vectorizing
for i, data in enumerate(valloader) :
    val_data = []
    inputs, labels = data
    for u in inputs :
        for col in u[0] :
            val_data += list(col)
    validation_vectorized.append(val_data)
    validation_labels.append([labels])
validation_vectorized = np.array(validation_vectorized)
validation_labels = np.array(validation_labels)

#### Section 3 END ####


# log variables setting : to record statements
log_training_loss = []
log_validation_loss = []
log_iter = []
log_training_acc = []
log_validation_acc = []



In [108]:
#### Section 4 START ####
# This Part includes declaring variables which will be used in train & predict & result visualization

# Initial Weight Value

# Learning rate 
learning_rate = 0.0002

i = 0

log_training_loss.append(1)

# Layer construct

# 2 hidden layer, 1 output layer
'''
Num Nodes in hidden layer : 4
w[1] (4, 1027, 10000, 1)
b[1] (4, 1027, 1)
a[1] (4, 1027, 1)

w[2] (4, 4, 1027, 1)
b[2] (4, 4, 1)

a[2] (4,  )


z = input_X * w[1][0~3] + b[1][0~3]


a[1] activation
(4, 1027, 1)

'''


input_X = training_vectorized
target_Y = training_labels




In [111]:
print(input_X.shape)
print(target_Y.shape)

(1027, 10000)
(1027, 1)


In [133]:
u = np.ones((10000,4))
v = np.ones((4,3))
w = np.ones((3,1))
a = np.ones((4,1))
b = np.ones((3,1))
c = np.ones((1,1))


def propagate(input_X,target_Y,u,v,w,a,b,c) :
    Loss = 0.0
    size = len(input_X)
    
    for x,t in zip(input_X,target_Y) :
        x = x.reshape((10000,1))
#        print("u.T:",(u.T).shape, "x:",x.shape,"a:",a.shape)
        
        z_for_p_i = np.dot(u.T,x)+a
        p_i = sigmoid(z_for_p_i) # 4,1
        
        #print("p_i:",p_i.shape)
        
        z_for_q_i = np.dot(v.T, p_i) + b
        q_i = sigmoid(z_for_q_i)
        #print("q_i:",q_i.shape) # 3,1
        z_for_y_h = np.dot(w.T,q_i) + c
        y_h = sigmoid(z_for_y_h)
        #print("y_h:",y_h.shape) # 1,1
        error = 0.5*((t - y_h)**2)
        w = w - learning_rate*np.dot(q_i,z_for_y_h)
        v = v - learning_rate*np.dot(p_i,z_for_q_i.T)
        u = u - learning_rate*np.dot(x,z_for_p_i.T)

        print(u,v,w)
    
# aE/ay^  = y^-t
# ay^/a(w.T*q_i + b) = y^(1-y^)
# a(w.T*q_i+b)/aw = q_i
# So, gredient decent for w is, 
# (y^-t)(y^(1-y^))(q_i)
# gredient decent for c is
# (y^-t)(y^(1-y^))()
    
        Loss += ((1/len(input_X)) * (-t* np.log(y_h) - (1-t) * np.log(1-y_h)))

propagate(input_X, target_Y, u,v,w,a,b,c)

    

[[ 0.14080969  0.14080969  0.14080969  0.14080969]
 [ 0.14080969  0.14080969  0.14080969  0.14080969]
 [ 0.14080969  0.14080969  0.14080969  0.14080969]
 ...
 [-0.28878541 -0.28878541 -0.28878541 -0.28878541]
 [-0.30009055 -0.30009055 -0.30009055 -0.30009055]
 [-0.26052258 -0.26052258 -0.26052258 -0.26052258]] [[0.999 0.999 0.999]
 [0.999 0.999 0.999]
 [0.999 0.999 0.999]
 [0.999 0.999 0.999]] [[0.99920934]
 [0.99920934]
 [0.99920934]]
[[ 0.17723272  0.17723272  0.17723272  0.17723272]
 [ 0.17699467  0.17699467  0.17699467  0.17699467]
 [ 0.17723272  0.17723272  0.17723272  0.17723272]
 ...
 [-0.23974526 -0.23974526 -0.23974526 -0.23974526]
 [-0.25200263 -0.25200263 -0.25200263 -0.25200263]
 [-0.21481525 -0.21481525 -0.21481525 -0.21481525]] [[0.999 0.999 0.999]
 [0.999 0.999 0.999]
 [0.999 0.999 0.999]
 [0.999 0.999 0.999]] [[0.99874272]
 [0.99874272]
 [0.99874272]]
[[ 0.1724559   0.1724559   0.1724559   0.1724559 ]
 [ 0.17221784  0.17221784  0.17221784  0.17221784]
 [ 0.1724559   0.1

[[ 0.16574627  0.16574627  0.16574627  0.16574627]
 [ 0.16851308  0.16851308  0.16851308  0.16851308]
 [ 0.16944061  0.16944061  0.16944061  0.16944061]
 ...
 [-0.2476859  -0.2476859  -0.2476859  -0.2476859 ]
 [-0.24597036 -0.24597036 -0.24597036 -0.24597036]
 [-0.2096187  -0.2096187  -0.2096187  -0.2096187 ]] [[0.94282716 0.94282716 0.94282716]
 [0.94282716 0.94282716 0.94282716]
 [0.94282716 0.94282716 0.94282716]
 [0.94282716 0.94282716 0.94282716]] [[0.92844149]
 [0.92844149]
 [0.92844149]]
[[ 0.16542815  0.16542815  0.16542815  0.16542815]
 [ 0.16818152  0.16818152  0.16818152  0.16818152]
 [ 0.1690956   0.1690956   0.1690956   0.1690956 ]
 ...
 [-0.24784496 -0.24784496 -0.24784496 -0.24784496]
 [-0.24610029 -0.24610029 -0.24610029 -0.24610029]
 [-0.20973295 -0.20973295 -0.20973295 -0.20973295]] [[0.94196353 0.94196353 0.94196353]
 [0.94196353 0.94196353 0.94196353]
 [0.94196353 0.94196353 0.94196353]
 [0.94196353 0.94196353 0.94196353]] [[0.92769789]
 [0.92769789]
 [0.92769789]]


[[ 0.17997336  0.17997336  0.17997336  0.17997336]
 [ 0.18669241  0.18669241  0.18669241  0.18669241]
 [ 0.19527715  0.19527715  0.19527715  0.19527715]
 ...
 [-0.21481332 -0.21481332 -0.21481332 -0.21481332]
 [-0.21764594 -0.21764594 -0.21764594 -0.21764594]
 [-0.19285615 -0.19285615 -0.19285615 -0.19285615]] [[0.88481648 0.88481648 0.88481648]
 [0.88481648 0.88481648 0.88481648]
 [0.88481648 0.88481648 0.88481648]
 [0.88481648 0.88481648 0.88481648]] [[0.85612167]
 [0.85612167]
 [0.85612167]]
[[ 0.17967348  0.17967348  0.17967348  0.17967348]
 [ 0.18631541  0.18631541  0.18631541  0.18631541]
 [ 0.19480876  0.19480876  0.19480876  0.19480876]
 ...
 [-0.21519317 -0.21519317 -0.21519317 -0.21519317]
 [-0.21798867 -0.21798867 -0.21798867 -0.21798867]
 [-0.19315603 -0.19315603 -0.19315603 -0.19315603]] [[0.88394944 0.88394944 0.88394944]
 [0.88394944 0.88394944 0.88394944]
 [0.88394944 0.88394944 0.88394944]
 [0.88394944 0.88394944 0.88394944]] [[0.85542211]
 [0.85542211]
 [0.85542211]]


[[ 0.148954    0.148954    0.148954    0.148954  ]
 [ 0.15839944  0.15839944  0.15839944  0.15839944]
 [ 0.16475962  0.16475962  0.16475962  0.16475962]
 ...
 [-0.17792093 -0.17792093 -0.17792093 -0.17792093]
 [-0.19131736 -0.19131736 -0.19131736 -0.19131736]
 [-0.16847103 -0.16847103 -0.16847103 -0.16847103]] [[0.82997735 0.82997735 0.82997735]
 [0.82997735 0.82997735 0.82997735]
 [0.82997735 0.82997735 0.82997735]
 [0.82997735 0.82997735 0.82997735]] [[0.79060016]
 [0.79060016]
 [0.79060016]]
[[ 0.14583267  0.14583267  0.14583267  0.14583267]
 [ 0.1552781   0.1552781   0.1552781   0.1552781 ]
 [ 0.16163829  0.16163829  0.16163829  0.16163829]
 ...
 [-0.18028335 -0.18028335 -0.18028335 -0.18028335]
 [-0.19363082 -0.19363082 -0.19363082 -0.19363082]
 [-0.17080897 -0.17080897 -0.17080897 -0.17080897]] [[0.82911337 0.82911337 0.82911337]
 [0.82911337 0.82911337 0.82911337]
 [0.82911337 0.82911337 0.82911337]
 [0.82911337 0.82911337 0.82911337]] [[0.7899408]
 [0.7899408]
 [0.7899408]]
[[ 

[[ 0.16378481  0.16378481  0.16378481  0.16378481]
 [ 0.17580669  0.17580669  0.17580669  0.17580669]
 [ 0.17820854  0.17820854  0.17820854  0.17820854]
 ...
 [-0.17877458 -0.17877458 -0.17877458 -0.17877458]
 [-0.18910498 -0.18910498 -0.18910498 -0.18910498]
 [-0.16326175 -0.16326175 -0.16326175 -0.16326175]] [[0.77986466 0.77986466 0.77986466]
 [0.77986466 0.77986466 0.77986466]
 [0.77986466 0.77986466 0.77986466]
 [0.77986466 0.77986466 0.77986466]] [[0.72677982]
 [0.72677982]
 [0.72677982]]
[[ 0.1639811   0.1639811   0.1639811   0.1639811 ]
 [ 0.17602858  0.17602858  0.17602858  0.17602858]
 [ 0.17856698  0.17856698  0.17856698  0.17856698]
 ...
 [-0.17834786 -0.17834786 -0.17834786 -0.17834786]
 [-0.1886868  -0.1886868  -0.1886868  -0.1886868 ]
 [-0.1628649  -0.1628649  -0.1628649  -0.1628649 ]] [[0.77986379 0.77986379 0.77986379]
 [0.77986379 0.77986379 0.77986379]
 [0.77986379 0.77986379 0.77986379]
 [0.77986379 0.77986379 0.77986379]] [[0.72639834]
 [0.72639834]
 [0.72639834]]


[[ 0.15199764  0.15199764  0.15199764  0.15199764]
 [ 0.16252247  0.16252247  0.16252247  0.16252247]
 [ 0.16480903  0.16480903  0.16480903  0.16480903]
 ...
 [-0.1727424  -0.1727424  -0.1727424  -0.1727424 ]
 [-0.17983258 -0.17983258 -0.17983258 -0.17983258]
 [-0.15554643 -0.15554643 -0.15554643 -0.15554643]] [[0.74112295 0.74112295 0.74112295]
 [0.74112295 0.74112295 0.74112295]
 [0.74112295 0.74112295 0.74112295]
 [0.74112295 0.74112295 0.74112295]] [[0.67270456]
 [0.67270456]
 [0.67270456]]
[[ 0.1511935   0.1511935   0.1511935   0.1511935 ]
 [ 0.16176443  0.16176443  0.16176443  0.16176443]
 [ 0.16400489  0.16400489  0.16400489  0.16400489]
 ...
 [-0.17302923 -0.17302923 -0.17302923 -0.17302923]
 [-0.18012965 -0.18012965 -0.18012965 -0.18012965]
 [-0.15587936 -0.15587936 -0.15587936 -0.15587936]] [[0.74033207 0.74033207 0.74033207]
 [0.74033207 0.74033207 0.74033207]
 [0.74033207 0.74033207 0.74033207]
 [0.74033207 0.74033207 0.74033207]] [[0.67211964]
 [0.67211964]
 [0.67211964]]


 [-0.15120732 -0.15120732 -0.15120732 -0.15120732]] [[0.70029965 0.70029965 0.70029965]
 [0.70029965 0.70029965 0.70029965]
 [0.70029965 0.70029965 0.70029965]
 [0.70029965 0.70029965 0.70029965]] [[0.61747928]
 [0.61747928]
 [0.61747928]]
[[ 0.15726111  0.15726111  0.15726111  0.15726111]
 [ 0.16748938  0.16748938  0.16748938  0.16748938]
 [ 0.17113498  0.17113498  0.17113498  0.17113498]
 ...
 [-0.17021931 -0.17021931 -0.17021931 -0.17021931]
 [-0.1765103  -0.1765103  -0.1765103  -0.1765103 ]
 [-0.15141413 -0.15141413 -0.15141413 -0.15141413]] [[0.69969848 0.69969848 0.69969848]
 [0.69969848 0.69969848 0.69969848]
 [0.69969848 0.69969848 0.69969848]
 [0.69969848 0.69969848 0.69969848]] [[0.61693755]
 [0.61693755]
 [0.61693755]]
[[ 0.15718903  0.15718903  0.15718903  0.15718903]
 [ 0.1674191   0.1674191   0.1674191   0.1674191 ]
 [ 0.1710647   0.1710647   0.1710647   0.1710647 ]
 ...
 [-0.17049865 -0.17049865 -0.17049865 -0.17049865]
 [-0.17680947 -0.17680947 -0.17680947 -0.17680947]


 [-0.15070129 -0.15070129 -0.15070129 -0.15070129]] [[0.65177566 0.65177566 0.65177566]
 [0.65177566 0.65177566 0.65177566]
 [0.65177566 0.65177566 0.65177566]
 [0.65177566 0.65177566 0.65177566]] [[0.56380648]
 [0.56380648]
 [0.56380648]]
[[ 0.15594067  0.15594067  0.15594067  0.15594067]
 [ 0.16869498  0.16869498  0.16869498  0.16869498]
 [ 0.17445587  0.17445587  0.17445587  0.17445587]
 ...
 [-0.17291777 -0.17291777 -0.17291777 -0.17291777]
 [-0.17753551 -0.17753551 -0.17753551 -0.17753551]
 [-0.1515859  -0.1515859  -0.1515859  -0.1515859 ]] [[0.65105712 0.65105712 0.65105712]
 [0.65105712 0.65105712 0.65105712]
 [0.65105712 0.65105712 0.65105712]
 [0.65105712 0.65105712 0.65105712]] [[0.56329125]
 [0.56329125]
 [0.56329125]]
[[ 0.15548279  0.15548279  0.15548279  0.15548279]
 [ 0.16823945  0.16823945  0.16823945  0.16823945]
 [ 0.17400033  0.17400033  0.17400033  0.17400033]
 ...
 [-0.17337095 -0.17337095 -0.17337095 -0.17337095]
 [-0.1779934  -0.1779934  -0.1779934  -0.1779934 ]


[[ 0.15161191  0.15161191  0.15161191  0.15161191]
 [ 0.16511431  0.16511431  0.16511431  0.16511431]
 [ 0.16917053  0.16917053  0.16917053  0.16917053]
 ...
 [-0.16971826 -0.16971826 -0.16971826 -0.16971826]
 [-0.17374076 -0.17374076 -0.17374076 -0.17374076]
 [-0.14776727 -0.14776727 -0.14776727 -0.14776727]] [[0.61496567 0.61496567 0.61496567]
 [0.61496567 0.61496567 0.61496567]
 [0.61496567 0.61496567 0.61496567]
 [0.61496567 0.61496567 0.61496567]] [[0.51403581]
 [0.51403581]
 [0.51403581]]
[[ 0.14952072  0.14952072  0.14952072  0.14952072]
 [ 0.16296393  0.16296393  0.16296393  0.16296393]
 [ 0.16696097  0.16696097  0.16696097  0.16696097]
 ...
 [-0.17158258 -0.17158258 -0.17158258 -0.17158258]
 [-0.17560508 -0.17560508 -0.17560508 -0.17560508]
 [-0.149602   -0.149602   -0.149602   -0.149602  ]] [[0.6142737 0.6142737 0.6142737]
 [0.6142737 0.6142737 0.6142737]
 [0.6142737 0.6142737 0.6142737]
 [0.6142737 0.6142737 0.6142737]] [[0.513552]
 [0.513552]
 [0.513552]]
[[ 0.14814716  0.1

In [ ]:
def backpropagate(input) :
    error = 0.5 * 

In [104]:
input_X = np.zeros((1027,10000))
w_1 = np.zeros((4,10000))
b_1 = np.zeros((4,1))

print(input_X.shape)
print(w_1.shape)
print((np.dot(input_X, w_1.T)).shape)
z = np.dot(input_X, w_1.T)
z = z.T
print(z.shape)

a_1 = sigmoid(z)
print(a_1.shape)


w_2 = np.zeros((5,1027))
b_2 = np.zeros((5,1))


z = np.dot(a_1,w_2.T)
print(z.shape)
a_2 = z.T
print(a_2.shape)

w_3 = 



#z_1 = np.dot(input_X, w_1) + b_1

    



(1027, 10000)
(4, 10000)
(1027, 4)
(4, 1027)
(4, 1027)
(4, 5)
(5, 4)


In [51]:
print(input_X.shape)
while (True):
    i += 1 
    output_Y = propagation(input_X, weight_list, hidden_layer_list)
    
    training_loss = lossFunction(output_Y,target_Y)
    training_acc = predict(output_Y, target_Y)
    print("iter : ",i, " t_loss : ",training_loss, " t_acc : ", training_acc)
    backpropagation(input_X, weight_list, hidden_layer_list, output_Y, target_Y, learning_rate)

    
    
    
    
print("Finished")    



(1027, 10000)
input_X.shape :  (1027, 10000)  weight_list[0].shape :  (3, 10000, 1)


ValueError: shapes (1027,1) and (10000,1) not aligned: 1 (dim 1) != 10000 (dim 0)